In [ ]:
import skimage
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.fftpack import fft2, ifft2

In [ ]:
# functions

def autocorrelation(x,intensity) :
    f = fft2(x)
    fConj = np.conj(f)
    p = f*fConj
    pi = ifft2(p)
    return np.fft.fftshift(np.real(pi)/((intensity**2)*len(x)*len(x[0])))-1

def correlationCorrection(x,corr) : 
    x[x < 0] = 0
    y  = x*(1/corr)*9 
    return y

def func(M,a,b,c) :
    x0 = xDim[indFitCrop[0]] # x and y values at peak
    y0 = yDim[indFitCrop[1]]
    x,y=M
    return a*np.exp(-((x-x0)**2 + (y-y0)**2)/c**2)+b

def fitCropping(z,h) :
    ind = np.unravel_index(np.argmax(z, axis=None), z.shape)
    if ind[0] > z.shape[0]-h or ind[1] > z.shape[1]-h or ind[0] < h or ind[1] < h:
        return np.zeros((2*h,2*h))
    else:
        return z[ind[0]-h:ind[0]+h,ind[1]-h:ind[1]+h]

###    
def image_stdev(region, intensities):
    # note the ddof arg to get the sample var if you so desire!
    return np.std(intensities[region], ddof=1)

In [ ]:
#constants for fitting
xDim = np.linspace(-(cropDim-1),cropDim,cropDim*2)*pixelSize #pixel size is dependent on image acquisition settings
yDim = np.linspace(-(cropDim-1),cropDim,cropDim*2)*pixelSize
X,Y=np.meshgrid(xDim,yDim)
xdata = np.vstack((X.ravel(), Y.ravel()))
initialGuess = (0.8, 0.14, 1.4)

In [ ]:
autoCorr = autocorrelation(regio,meanInt) # run correlation -- region is the region of interest

autoCorrCorrected = correlationCorrection(autoCorr, extent) # correct correlation
    
fitCrop = fitCropping(autoCorrCorrected,cropDim) # crop autocorrelation for fitting
indFitCrop = np.unravel_index(np.argmax(fitCrop, axis=None), fitCrop.shape)#find peak in cropped image
popt,pcov = opt.curve_fit(func,xdata,fitCrop.ravel(), p0 = initialGuess) # fit

# pull out parameters
perr = np.sqrt(np.diag(pcov)) # stdev of fits
nop = 1/popt[0]
w0 = popt[2]
DA = meanInt/nopAb1 # meanInt is the region intensity found from other functions, such as regionprops
